In [3]:
## CELL 1: LOAD ###
# run this cell to load your datafile through the GUI (currently only works with txt, in Radars template)
# outputs a 0 if there's an error loading the file

from miniMEAPsys import loadem
df,file_stat=loadem()
print(file_stat)

/home/nmd/l/Caitlin_TREV/miniMEAP/sj06_RADARS.txt successfully loaded


In [9]:
## CELL 2: OPEN THE INTERACTIVE PLOT ###
# run this cell to open the interactive plot to visualise and amend peaks

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
from miniMEAPsys import *

%matplotlib tk
plt.close('all')
fig,ax = plt.subplots()
plt.subplots_adjust(bottom=0.25)

t,s,peak_times,peak_vals = get_ts(df)

w = ax.plot(t,s)
peak_plot, = ax.plot(peak_times,peak_vals ,'vm')

# Set the axis and slider position in the plot
bottom_pos = plt.axes([0.2, 0.1, 0.65, 0.03],facecolor = 'white')
scroll_slider = Slider(bottom_pos,'time', -1,t[-1])

# Make a vertically oriented slider to control the amplitude
right_pos = plt.axes([.95, 0.25, 0.0225, 0.63])
y_zoom = Slider(right_pos,label="Y zoom",valmin=0,valmax=1,valinit=.5,orientation="vertical")
left_pos = plt.axes([.05, 0.25, 0.0225, 0.63])
x_zoom = Slider(left_pos,label="X zoom",valmin=0,valmax=1,valinit=.5,orientation="vertical")

ax.set_xlim(-1,4)

def update(val):
    pos = scroll_slider.val
    yzoom = y_zoom.val
    xzoom = x_zoom.val
    ax.set_xlim(pos, pos+10*xzoom)
    fig.canvas.draw_idle
    ax.set_ylim(-yzoom, yzoom)
    fig.canvas.draw_idle
    
# update function called using on_changed() function
scroll_slider.on_changed(update)
y_zoom.on_changed(update)
x_zoom.on_changed(update)

# Display the plot
plt.show()

def onclick(event):
    global peak_times,peak_vals,fig,peak_plot
    ix, iy, ib, ik = event.xdata, event.ydata, event.button, event.key
    if (ix!=None):
        if (np.min(np.abs(ix-peak_times))<100) & (event.button!=1) & (ik==None):
            peak_times,peak_vals,peak_plot=remove_point(ix,peak_times,peak_vals,peak_plot)
        elif (event.button==1) & (ik==None):
            peak_times,peak_vals,peak_plot=add_point(ix,iy,peak_times,peak_vals,peak_plot)
        elif (ik=='m'):
            peak_times,peak_vals,peak_plot=meap_dat(ix,iy,peak_times,peak_vals,peak_plot)
        fig.canvas.draw()
        fig.canvas.flush_events()
        
        if not plt.fignum_exists(1):
            canvas.mpl_disconnect(cid)
            canvas.mpl_disconnect(cidk)
            
    return peak_times,peak_vals

def onarrow(e):
    global scroll_slider
    if e.key == "right":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]+.500)
    elif e.key == "left":
        pos = ax.get_xlim()
        scroll_slider.set_val(pos[0]-.500)
    else:
        return

cid = fig.canvas.mpl_connect('button_press_event', onclick)
cidk = fig.canvas.mpl_connect('key_press_event', onarrow)